In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_csv('melb_data.csv')

data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [4]:
# Our target
y = data.Price

# Things we will use to predict the price
price_predictors = data.drop(['Price'], axis='columns')

# Taking only numerical predictors
X = price_predictors.select_dtypes(exclude = ['object'])

# Splitting our data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size = 0.8, test_size= 0.2)



In [5]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer()

# Get columns with missing values

emptyCols = []

for col in X_train.columns:
    # Returns true if there is at least one missing value
    if X_train[col].isnull().any():
        emptyCols.append(col)

imputed_X_train = pd.DataFrame(imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(imputer.transform(X_valid))

# Imputation removes column names
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns


In [6]:
# Now we will get categorical values
# We are assuming that all columns with text are categorical values
# To get categorical columns, we are going to select the columns with low cardinality (low number of unique values)


X = price_predictors
X_train_full, X_valid_full, y_train_full, y_valid_full = train_test_split(X, y, train_size=0.8, test_size=0.2)


missing_values_columns = []

for col in X_train_full.columns:
    if X_train_full[col].isnull().any():
        missing_values_columns.append(col)

X_train_full = X_train_full.drop(missing_values_columns, axis = 1)
X_valid_full = X_valid_full.drop(missing_values_columns, axis = 1)

low_cardinality_columns = []

for col in X_train_full.columns:
    if X_train_full[col].nunique() < 10 and X_train_full[col].dtype == 'object':
        low_cardinality_columns.append(col)

print(low_cardinality_columns)

numerical_cols = []

for col in X_train_full.columns:
    if X_train_full[col].dtype in ['int64', 'float64']:
        numerical_cols.append(col)

print(numerical_cols)

required_columns = low_cardinality_columns + numerical_cols

# We have successfully isolated X_train to contain only categorical values and numerical values
X_train = X_train_full[required_columns].copy()
X_valid = X_valid_full[required_columns].copy()

X_train.head()

['Type', 'Method', 'Regionname']
['Rooms', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Landsize', 'Lattitude', 'Longtitude', 'Propertycount']


,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount
12716,t,VB,Western Metropolitan,3,7.5,3040.0,3.0,1.0,116.0,-37.73899,144.89291,9264.0
12563,u,SP,Eastern Metropolitan,2,10.2,3127.0,2.0,1.0,2243.0,-37.81452,145.10884,2079.0
11413,h,S,Eastern Metropolitan,4,23.3,3180.0,4.0,2.0,633.0,-37.88214,145.25058,2949.0
5181,u,VB,Northern Metropolitan,3,11.2,3073.0,3.0,1.0,179.0,-37.71080,145.02890,21650.0
10690,h,S,Northern Metropolitan,3,11.2,3046.0,3.0,1.0,626.0,-37.71425,144.95013,2606.0


In [7]:
# Performing One-Hot encoding of categorical values
from sklearn.preprocessing import OneHotEncoder

categorical_columns = []

for cols in X_train.columns:
    if (X_train[cols].dtype == 'object'):
        categorical_columns.append(cols)

# sparse = false so that the output comes as a numpy array and not as a sparse matrix
# handle_unknown so that we can ignore erros in case the encoder comes across classes that arent in the training data
OH_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Getting the onehot encoder columns
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid))

# One-hot encoding removed index, add indexes
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove the categorical columns
num_X_train = X_train.drop(categorical_columns, axis = 1)
num_X_valid = X_valid.drop(categorical_columns, axis = 1)

# Concatenate the one-hot encoded columnns to the dataframe
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis = 1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis = 1)

# Ensure all columns have string type
OH_X_train.columns = OH_X_train.columns.astype(str)
OH_X_valid.columns = OH_X_valid.columns.astype(str)



In [8]:
# Separate target from predictors
y = data.Price
X = data.drop(['Price'], axis=1)

# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])


In [10]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

In [18]:

# Select subset of predictors
cols_to_use = ['Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt']
X = data[cols_to_use]

# Select target
y = data.Price

# Separate data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y)


In [25]:
from xgboost import XGBRegressor

# Using Gradient Boosting

xgboost_model = XGBRegressor(n_estimators = 1000, learning_rate = 0.05, n_jobs = 4)

xgboost_model.fit(X_train, y_train, early_stopping_rounds=5, eval_set=[(X_valid, y_valid)],
             verbose=False)

/Users/aadeesh/miniconda3/envs/titanic/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)